In [4]:
import folium
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely import wkt

In [2]:
def prepare_data(pid:str):    
    #load all data
    df = pd.read_csv("../../test_full_pattern.csv")
    # pick one trip to work with
    
    trip_id = df["unique_trip_vehicle_day"][0]

    one_trip_df = df[df["unique_trip_vehicle_day"]== trip_id]

    one_trip_df['location'] = one_trip_df['location'].astype(str)
    one_trip_df['geometry'] = one_trip_df['location'].apply(wkt.loads)

    one_trip_df = gpd.GeoDataFrame(one_trip_df, geometry='geometry', crs='epsg:4326')

    bus = one_trip_df[one_trip_df['typ'] == 'B']
    bus = gpd.GeoSeries(bus["geometry"])
    bus = bus.to_json()

    stop = one_trip_df[one_trip_df['typ'] == 'S']
    stop = gpd.GeoSeries(stop["geometry"])
    stop = stop.to_json()

    other = one_trip_df[one_trip_df['typ'] == 'W']
    other = gpd.GeoSeries(other["geometry"])
    other = other.to_json()

    segments_df = pd.read_parquet(f"../cta-stop-etl/out/patterns/pid_{pid}_segment.parquet")
    segments_df['geometry'] = segments_df['geometry'].apply(wkt.loads)
    segments_df = gpd.GeoDataFrame(segments_df, crs='EPSG:4326')

    pattern_segments = gpd.GeoSeries(segments_df["geometry"])
    pattern_segments = pattern_segments.to_json()

    return bus, stop, other, pattern_segments

In [22]:
df[df['rt'] == '93']['pid'].value_counts()

pid
1802.0    513
1800.0    491
8140.0     16
5736.0     13
1798.0     11
1799.0      9
5376.0      3
Name: count, dtype: int64

In [5]:
# Choose a route you want to find  
rt = '29'

df = pd.read_parquet("../cta-stop-etl/out/parquets/2023-11-16.parquet")
#just chooses one of the patterns from this route
pid = int(df[df['rt']==rt]['pid'].unique()[0])
pid


1039

In [135]:
#then run 
# python -m cta-stop-watch.cta-stop-etl.calculate_stop_time {pid} 100
# to get the bus location and stop times for the pattern (just looks at 100 trips)

#then run and the map below
bus, stop, other, pattern_segments = prepare_data(1799)

/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_56825/1655922356.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_trip_df['location'] = one_trip_df['location'].astype(str)
/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_56825/1655922356.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_trip_df['geometry'] = one_trip_df['location'].apply(wkt.loads)


In [136]:
m = folium.Map(
    location=[41.83491987636846, -87.62004994799588],
    zoom_start=12,
    zoomSnap=0.5,
    tiles=None,
    min_zoom=10,
    max_zoom=12,
    min_lat=41.6,
    max_lat=42.1,
    min_lon=-87.27481026390364,
    max_lon=-87.981026390364,
    max_bounds=True,
)
base_map = folium.FeatureGroup(name="Basemap", overlay=True, control=False)
folium.TileLayer(tiles="cartodb positron").add_to(base_map)
base_map.add_to(m)


# add one trip bus location
folium.GeoJson(data=bus, marker=folium.Circle(
                   radius=50,
                   fill_color="red",
                   fill_opacity=0.4,
                   color="black",
                   weight=1), 
                   name = 'One Bus Trip').add_to(m)

# add bus stops
folium.GeoJson(data=stop, marker=folium.Circle(
                   radius=30,
                   fill_color="green",
                   fill_opacity=0.4,
                   color="black",
                   weight=1),
                    name = 'Bus Stops').add_to(m)

folium.GeoJson(data=other, marker=folium.Circle(
                   radius=10,
                   fill_color="blue",
                   fill_opacity=0.4,
                   color="black",
                   weight=1),
                    name = 'Other Bus Routes Waypoints').add_to(m)



# add segments
folium.GeoJson(data=pattern_segments, style_function=lambda x: {"fillColor": "grey", "weight": '1'}, name = 'Route with Buffer').add_to(m)

folium.LayerControl().add_to(m)

m

In [14]:
#

segments = [1800., 1798., 1802., 5736., 1799., 8140., 5376.]


patterns_json = []
for seg in segments:
    seg = int(seg)
    segments_df = pd.read_parquet(f"../cta-stop-etl/out/patterns/pid_{seg}_segment.parquet")
    segments_df['geometry'] = segments_df['geometry'].apply(wkt.loads)
    segments_df = gpd.GeoDataFrame(segments_df, crs='EPSG:4326')

    pattern_segments = gpd.GeoSeries(segments_df["geometry"])
    pattern_segments = pattern_segments.to_json()
    patterns_json.append(pattern_segments)





In [27]:
# weird example of all the patterns for a segments can just be parts
segments = [1800., 1798., 1802., 5736., 1799., 8140., 5376.]


patterns_json = []
for seg in segments:
    seg = int(seg)
    segments_df = pd.read_parquet(f"../cta-stop-etl/out/patterns/pid_{seg}_segment.parquet")
    segments_df['geometry'] = segments_df['geometry'].apply(wkt.loads)
    segments_df = gpd.GeoDataFrame(segments_df, crs='EPSG:4326')

    pattern_segments = gpd.GeoSeries(segments_df["geometry"])
    pattern_segments = pattern_segments.to_json()
    patterns_json.append(pattern_segments)



colors = [
    'red',
    'blue',
    'gray',
    'orange',
    'beige',
    'green',
    'pink'
]
segments = [1800., 1798., 1802., 5736., 1799., 8140., 5376.]

m = folium.Map(
    location=[41.83491987636846, -87.62004994799588],
    zoom_start=12,
    zoomSnap=0.5,
    tiles=None,
    min_zoom=10,
    max_zoom=12,
    min_lat=41.6,
    max_lat=42.1,
    min_lon=-87.27481026390364,
    max_lon=-87.981026390364,
    max_bounds=True,
)
base_map = folium.FeatureGroup(name="Basemap", overlay=True, control=False)
folium.TileLayer(tiles="cartodb positron").add_to(base_map)
base_map.add_to(m)

for i, pattern in enumerate(patterns_json):
    print(colors[i])
    folium.GeoJson(data=pattern, style_function=lambda x: {"fillColor":colors[i],"color":colors[i], "weight": '1'}, name = str(segments[i])).add_to(m)

folium.LayerControl().add_to(m)

m

red
blue
gray
orange
beige
green
pink
